In [1]:
from mode import *

In [2]:
keys = [np.random.randint(0, 2**64, dtype=np.uint64) for _ in range(2)]
print("Keys:")
print(f"{keys[0]:x}")
print(f"{keys[1]:x}")


Keys:
43ca6b6dcd1464ab
334336650bb489e2


In [3]:
message = int(''.join([str(np.random.randint(0, 2**64, dtype=np.uint64)) for _ in range(16)]))
print("Message:", f"{message:x}")

aux, cipher, tag = encrypt_and_authenticate(keys, message)
print("Cipher:", f"{cipher:x}")

#cipher += 1

verified, decrypted = decrypt_and_verify(keys, cipher, tag, aux)
print("Verified:", verified)
if verified:
    print("Decrypted:", f"{decrypted:x}")


Message: 99061697f78c4f8ba677ae730efd5c24ce22979b0ca22ebbbde0c6627726b92468cd7847ab3168ccac07c38c50d77513de9ab1b80dd00a30c0cef0997d08ccb5de9593afb2e57bf43c100237b47b8680c40c6392403b954986228383c1d72be6d06cb23ee9a55d4a0f286c3bce20067703a4885ce01972ec613c298627a454bb33b82
Cipher: ed18c6f17bceec1cf1ab270ef9387f5e3959d95f7fb443a91036a0e4ba6533a10c0ec941bc2a8d78d05577f9b41d4dc3d6e18633fef444a67a05922bec174068a34c7842211ade8630e0c1db853a77afdc1b6d454a0a4f2eebd37480c23a6e2c85c2a1792bcc3576d9400f0860eab05242793f5a67f77285e9372eb4bb917ad545e25e3101f94adb
Verified: True
Decrypted: 99061697f78c4f8ba677ae730efd5c24ce22979b0ca22ebbbde0c6627726b92468cd7847ab3168ccac07c38c50d77513de9ab1b80dd00a30c0cef0997d08ccb5de9593afb2e57bf43c100237b47b8680c40c6392403b954986228383c1d72be6d06cb23ee9a55d4a0f286c3bce20067703a4885ce01972ec613c298627a454bb33b82


/Users/neosapien/Development/exam-sg8/sbox.py:24: RuntimeWarning: overflow encountered in scalar multiply
  x *= b
/Users/neosapien/Development/exam-sg8/sbox.py:27: RuntimeWarning: overflow encountered in scalar multiply
  x *= a
/Users/neosapien/Development/exam-sg8/sbox.py:33: RuntimeWarning: overflow encountered in scalar multiply
  x *= a_inv
/Users/neosapien/Development/exam-sg8/sbox.py:36: RuntimeWarning: overflow encountered in scalar multiply
  x *= b_inv


In [8]:
from hash16 import hash

a = message
b = message + 2**16

print(f"{hash(a):x}")
print(f"{hash(b):x}")

e48657b4851afdf5377c0bd5138afeaa
e48657b4851afdf5377c0bd56bc9feaa


In [6]:
import random
from hash16 import *  

def random_walk_hash(start, steps=1000):
    current = start
    print(f"Départ : {hex(current)}")
    for i in range(steps):
        # Mar​che aléatoire : flip 1 bit aléatoire
        bit_to_flip = 1 << random.randint(0, 127)
        current ^= bit_to_flip
        h = hash(current)
        print(f"Étape {i+1:4d} : message = {hex(current)} | hash = {hex(h)}")

start_msg = random.getrandbits(128)
random_walk_hash(start_msg, steps=50)


Départ : 0xd8a8d71ac1b57aa8ce3edfff817e7cad
Étape    1 : message = 0xd8a8d71ec1b57aa8ce3edfff817e7cad | hash = 0x6338e64418551fa66c98dff592636115
Étape    2 : message = 0xd8a8d71ec1b57aa8ce3adfff817e7cad | hash = 0x6338e64418551fa6acc2dff592636115
Étape    3 : message = 0xd888d71ec1b57aa8ce3adfff817e7cad | hash = 0x6306e64418551fa6acc2dff592636115
Étape    4 : message = 0xd898d71ec1b57aa8ce3adfff817e7cad | hash = 0x6337e64418551fa6acc2dff592636115
Étape    5 : message = 0xd998d71ec1b57aa8ce3adfff817e7cad | hash = 0x6367e64418551fa6acc2dff592636115
Étape    6 : message = 0xd998d71e81b57aa8ce3adfff817e7cad | hash = 0x6367e644c40f1fa6acc2dff592636115
Étape    7 : message = 0xd998d71e81b57aa8ce3addff817e7cad | hash = 0x6367e644c40f1fa6acc2df9592636115
Étape    8 : message = 0xd998d51e81b57aa8ce3addff817e7cad | hash = 0x6367e624c40f1fa6acc2df9592636115
Étape    9 : message = 0xd998d51e81b57aa8de3addff817e7cad | hash = 0x6367e624c40f1fa6b582df9592636115
Étape   10 : message = 0xd998d51e81b57

/Users/LeroyCapucine/Desktop/CS_cours/2A/crypto/exam-sg8/sbox.py:24: RuntimeWarning: overflow encountered in scalar multiply
  x *= b
/Users/LeroyCapucine/Desktop/CS_cours/2A/crypto/exam-sg8/sbox.py:27: RuntimeWarning: overflow encountered in scalar multiply
  x *= a
